<a href="https://colab.research.google.com/github/Rocking-Priya/Mod-C-Summer-2025-Week-1--6-assignment/blob/main/Mod_C_Week_1_%E2%80%93_Linear_Regression_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TASK FOR WEEK 1 -  Include concepts such as linear regression with polynomial terms, interaction terms, multicollinearity, variance inflation factor and regression, and categorical and continuous features.

In [3]:
import pandas as pd

In [4]:
phishing_df_1 = pd.read_csv('/content/dataset_phishing.csv')
phishing_df_1.head()

,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,37,19,0,3,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,77,23,1,1,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,126,50,1,4,1,0,1,2,0,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,18,11,0,2,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,55,15,0,2,2,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [5]:
phishing_df_1.isnull().sum()

,0
url,0
length_url,0
length_hostname,0
ip,0
nb_dots,0
...,...
web_traffic,0
dns_record,0
google_index,0
page_rank,0


In [7]:
X = phishing_df_1.drop('status', axis=1)
y = phishing_df_1['status']

In [8]:
# Check data types of columns in X
print("Data types of X before VIF calculation:")
print(X.dtypes)

Data types of X before VIF calculation:
url                object
length_url          int64
length_hostname     int64
ip                  int64
nb_dots             int64
                    ...  
domain_age          int64
web_traffic         int64
dns_record          int64
google_index        int64
page_rank           int64
Length: 88, dtype: object


In [4]:
import numpy as np

In [9]:
X_numeric = X.select_dtypes(include=np.number)

In [17]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [11]:
X_with_const = add_constant(X_numeric) # Use the numeric DataFrame. This adds a column of 1s (called "const") to your data so the model includes an intercept term.

vif = pd.DataFrame()
vif["Variable"] = X_with_const.columns #This part calculates the VIF for each variable (column).

# Calculate VIF, ensuring we use the numeric values
vif["VIF"] = [variance_inflation_factor(X_with_const.values, i) for i in range(X_with_const.shape[1])] #VIF shows how much a variable is related to other variables.
vif

/usr/local/lib/python3.11/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


,Variable,VIF
0,const,124.747685
1,length_url,11.469510
2,length_hostname,3.941609
3,ip,3.413926
4,nb_dots,8.152036
...,...,...
83,domain_age,1.875603
84,web_traffic,1.244188
85,dns_record,1.652678
86,google_index,1.967224


VIF > 10 means strong multicollinearity. That variable might be too similar to others and could mess up your model.

Example: length_url has a VIF of 11.47, so it might be giving similar information as other variables. You might consider removing or combining it with related features.

High VIF means the model might not be trustworthy.

Multicollinearity can make it hard to tell which variable is really important.



In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


# Convert the target variable 'y' to numerical values
# Map 'phishing' to 1 and 'legitimate' to 0 (or vice versa)
y_encoded = y.map({'phishing': 1, 'legitimate': 0})

# Select only numeric columns for X
# Keep this as before, ensuring only numeric features are used for the model
X_numeric = X.select_dtypes(include=np.number)

# Split data using the numeric DataFrame for X and the encoded target variable y
X_train, X_test, y_train, y_test = train_test_split(X_numeric, y_encoded, test_size=0.2, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print("R² score:", r2_score(y_test, y_pred))

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)

# Calculate RMSE by taking the square root of the MSE
print("RMSE:", np.sqrt(mse))

R² score: 0.7442130858002536
RMSE: 0.25285793430805237


You've trained a linear regression model to predict whether a website is phishing or legitimate, based on the numeric features of the website. You've evaluated its performance using two metrics: R-squared (R²) and Root Mean Squared Error (RMSE).

Here's what those values mean:

R² score: 0.7442130858002536

The R-squared value, also known as the coefficient of determination, represents the proportion of the variance in the dependent variable (whether the website is phishing or legitimate) that is predictable from the independent variables (the numeric features).
An R² of 0.744 (or approximately 74.4%) means that your model explains about 74.4% of the variability in the target variable.
In simpler terms, your model does a reasonably good job of capturing the relationship between the website features and its status (phishing or legitimate). A higher R² generally indicates a better fit of the model to the data, though a high R² doesn't necessarily mean the model is perfect or that the variables are the only factors.
RMSE: 0.25285793430805237

RMSE stands for Root Mean Squared Error. It measures the average magnitude of the errors between the predicted values and the actual values.
The unit of RMSE is the same as the unit of the dependent variable. In your case, your dependent variable y_encoded is either 0 (legitimate) or 1 (phishing).
An RMSE of approximately 0.253 means that, on average, your model's predictions are about 0.253 units away from the actual value (0 or 1).
Since your target variable is binary (0 or 1), an RMSE of 0.253 indicates that the model's predictions, which are continuous values between 0 and 1, deviate from the true binary status by this amount on average. A lower RMSE indicates that the model's predictions are closer to the actual values, which is desirable.
In summary:

Your model is able to explain a significant portion (about 74.4%) of the variation in whether a website is phishing or not based on its numeric features.
The average error in your model's predictions, in terms of the 0/1 scale, is about 0.253.
These metrics suggest that your linear regression model has some predictive power for classifying websites as phishing or legitimate based on the numeric features, although it's not a perfect prediction. Since your target variable is binary, it's worth noting that while you've used a linear regression model (which is typically for continuous outcomes), the results are interpreted in the context of predicting a value between 0 and 1. For binary classification problems, metrics like accuracy, precision, recall, F1-score, and ROC AUC are often more appropriate, and models designed for classification (like Logistic Regression, Support Vector Machines, or Tree-based models) are typically used.

In [15]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)

# Select only numeric columns from X before applying PolynomialFeatures
X_numeric = X.select_dtypes(include=np.number)
X_poly = poly.fit_transform(X_numeric)

# Convert the target variable 'y' to numerical values
# Map 'phishing' to 1 and 'legitimate' to 0 (or vice versa)
y_encoded = y.map({'phishing': 1, 'legitimate': 0})

# Train model again, using the polynomial features and encoded target variable
X_train_poly, X_test_poly, y_train, y_test = train_test_split(X_poly, y_encoded, test_size=0.2, random_state=42)
model_poly = LinearRegression()
model_poly.fit(X_train_poly, y_train)

# Predict and evaluate
y_pred_poly = model_poly.predict(X_test_poly)
print("R² (Poly):", r2_score(y_test, y_pred_poly))
# Calculate MSE
mse_poly = mean_squared_error(y_test, y_pred_poly)
# Calculate RMSE by taking the square root of the MSE
print("RMSE (Poly):", np.sqrt(mse_poly))

R² (Poly): 0.7085155193388634
RMSE (Poly): 0.2699262634055965


In [6]:
phish_email_df = pd.read_csv('/content/Phishing_Email.csv')
phish_email_df.head()

,Unnamed: 0,Email Text,Email Type
0,0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,1,the other side of * galicismos * * galicismo *...,Safe Email
2,2,re : equistar deal tickets are you still avail...,Safe Email
3,3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email
4,4,software at incredibly low prices ( 86 % lower...,Phishing Email


In [7]:
# Rename for clarity
df = phish_email_df.rename(columns={"Email Text": "text", "Email Type": "label"})

# Check first few rows
df.head()

,Unnamed: 0,text,label
0,0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,1,the other side of * galicismos * * galicismo *...,Safe Email
2,2,re : equistar deal tickets are you still avail...,Safe Email
3,3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email
4,4,software at incredibly low prices ( 86 % lower...,Phishing Email


In [8]:
# Map 'Phishing Email' to 1, 'Safe Email' to 0
df['label_encoded'] = df['label'].map({'Phishing Email': 1, 'Safe Email': 0})


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Handle missing values in the 'text' column by replacing them with an empty string
df['text'] = df['text'].fillna('')

# Turn text into numeric vectors
vectorizer = TfidfVectorizer(max_features=500)  # limit to 500 features
X = vectorizer.fit_transform(df['text']).toarray()

# Target variable
y = df['label_encoded']

In [22]:

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


# Add constant for intercept
X_const = add_constant(X)

# Create DataFrame to store VIF values
vif_df = pd.DataFrame()
vif_df["VIF"] = [variance_inflation_factor(X_const, i) for i in range(X_const.shape[1])]
vif_df["Variable"] = ["const"] + list(vectorizer.get_feature_names_out())

# Show top 10 variables with highest VIF
vif_df.sort_values("VIF", ascending=False).head(10)


,VIF,Variable
251,18.224335,listinfo
261,14.705512,mailman
0,12.532452,const
202,6.472507,hou
144,6.431540,ect
207,5.906680,ie
249,5.906325,linux
252,5.601422,lists
206,3.350002,http
425,2.786898,the


In [24]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print("R² score:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

R² score: 0.7130561465783966
RMSE: 0.2613479663625455


In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)

# Split and train as before
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate
print("R² (Poly):", r2_score(y_test, y_pred))
print("RMSE (Poly):", np.sqrt(mean_squared_error(y_test, y_pred)))

In [10]:
df_3 = pd.read_csv('/content/2022-12-09-enriched.csv')
df_3.head()

,cve_id,vendor_project,product,vulnerability_name,date_added,short_description,required_action,due_date,notes,grp,pub_date,cvss,cwe,vector,complexity,severity
0,CVE-2021-27104,accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA 9_12_370 and earlier is affected...,Apply updates per vendor instructions.,2021-11-17,NaN,1,2021-02-16,9.8,CWE-78,NETWORK,LOW,CRITICAL
1,CVE-2021-27102,accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA 9_12_411 and earlier is affected...,Apply updates per vendor instructions.,2021-11-17,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
2,CVE-2021-27101,accellion,FTA,Accellion FTA SQL Injection Vulnerability,2021-11-03,Accellion FTA 9_12_370 and earlier is affected...,Apply updates per vendor instructions.,2021-11-17,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
3,CVE-2021-27103,accellion,FTA,Accellion FTA SSRF Vulnerability,2021-11-03,Accellion FTA 9_12_411 and earlier is affected...,Apply updates per vendor instructions.,2021-11-17,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
4,CVE-2021-21017,adobe,Acrobat and Reader,Adobe Acrobat and Reader Heap-based Buffer Ove...,2021-11-03,Acrobat Reader DC versions versions 2020.013.2...,Apply updates per vendor instructions.,2021-11-17,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Drop columns with too many missing values or that aren't useful for regression
df_3_cleaned = df_3.drop(columns=['cve_id', 'notes', 'short_description', 'required_action', 'vulnerability_name'])


In [13]:
# Drop rows where CVSS score is missing (we're predicting this!)
df = df_3_cleaned.dropna(subset=['cvss'])

# Fill missing categorical values with 'Unknown'
df3 = df_3_cleaned.fillna('Unknown')


In [14]:
df_encoded = pd.get_dummies(df3, columns=['vendor_project', 'product', 'complexity', 'severity', 'vector', 'cwe'])


In [15]:
# Define X and y
X = df_encoded.drop(columns=['cvss'])  # Features
y = df_encoded['cvss']                 # Target: CVSS score


In [19]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [21]:
# Drop date columns as they are not suitable for VIF calculation in their current format
X_numeric_for_vif = X.drop(columns=['date_added', 'due_date', 'pub_date'])

# Ensure all remaining columns are numeric by selecting only numeric types
X_numeric_for_vif = X_numeric_for_vif.select_dtypes(include=np.number)

# Add constant
X_const = add_constant(X_numeric_for_vif)

# Calculate VIF
vif = pd.DataFrame()
vif["Variable"] = X_const.columns
vif["VIF"] = [variance_inflation_factor(X_const.values, i) for i in range(X_const.shape[1])]

# Show high VIF values
vif.sort_values(by='VIF', ascending=False).head(10)

,Variable,VIF
0,const,2.116936
1,grp,1.000000


In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Create a new DataFrame that contains only numeric values for X and y
# Drop rows where 'cvss' is 'Unknown' to ensure y is numeric
df_numeric_model = df_encoded[df_encoded['cvss'] != 'Unknown'].copy()

# Define X and y from the new numeric DataFrame
X_numeric = df_numeric_model.drop(columns=['cvss', 'date_added', 'due_date', 'pub_date'])
y_numeric = df_numeric_model['cvss'].astype(float) # Ensure y is float

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_numeric, y_numeric, test_size=0.2, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print("R² score:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

R² score: 0.6124094669018225
RMSE: 0.5869624505106705


In [27]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Create a new DataFrame that contains only numeric values for X and y
# Drop rows where 'cvss' is 'Unknown' to ensure y is numeric
df_numeric_model = df_encoded[df_encoded['cvss'] != 'Unknown'].copy()

# Define X and y from the new numeric DataFrame
X_numeric = df_numeric_model.drop(columns=['cvss', 'date_added', 'due_date', 'pub_date'])
y_numeric = df_numeric_model['cvss'].astype(float) # Ensure y is float


# Create polynomial features from the numeric features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_numeric)


# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_poly, y_numeric, test_size=0.2, random_state=42)

# Train regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("R² (Poly):", r2_score(y_test, y_pred))
print("RMSE (Poly):", np.sqrt(mean_squared_error(y_test, y_pred)))

R² (Poly): -0.9539194173737806
RMSE (Poly): 1.3178836291144247
